In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1012. Allostatic Load/P1012.3. Analyses/P1012.3.2. Ordinal Regression/Output')
cur_date = "052724"

library(readxl)
library(openxlsx)
library(tidyverse)
library(ordinalNet)

# reading in file
bp_df = data.frame(read_excel("Input/Allostatic_Mediator_Data_050824.xlsx", sheet = 4)) 
subject_info_df = data.frame(read_excel("Input/Subject_Info_050824.xlsx", sheet = 2))
mediator_scores_df = data.frame(read_excel("Input/Mediator_Scores_BP_Subjects_052724.xlsx"))

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.3     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
head(bp_df)
head(subject_info_df)
head(mediator_scores_df)

,Subject_ID,Systolic,Diastolic,BP_Classification
,<chr>,<dbl>,<dbl>,<chr>
1,CS_26,133,86,1
2,CS_29,116,84,N
3,CS_17,118,80,N
4,NS_63,122,85,E
5,CS_21,112,77,N
6,CS_27,131,91,1


,Study,Original_Subject_Number,Subject_Number,Group,Subject_ID,Sex,Age,Race
,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>
1,FM2,97,1,CS,CS_1,M,18,B
2,FM2,99,2,CS,CS_2,F,20,W
3,FM2,101,3,CS,CS_3,F,20,W
4,FM2,98,4,CS,CS_4,F,21,W
5,TCORS LAIV,39,5,CS,CS_5,M,21,W
6,FM2,100,6,CS,CS_6,F,22,W


,Variable,Subject_ID,Mediator_Score
,<chr>,<chr>,<dbl>
1,Noradrenaline,CS_5,0.9209663
2,Noradrenaline,CS_8,0.5018144
3,Noradrenaline,CS_10,0.8466564
4,Noradrenaline,CS_16,0.5018144
5,Noradrenaline,CS_17,0.3297612
6,Noradrenaline,CS_21,0.2711344


In this analysis, we're using [ordinal regression](https://www.statisticssolutions.com/free-resources/directory-of-statistical-analyses/ordinal-regression/) to predict blood pressure classification using mediator scores from 17 biomarkers. 

In [3]:
# creating a vector for variables that increase atherosclerotic risk
# HDL decreases risk
#allostatic_load_biomarkers = c('Cortisol','Noradrenaline','Epinephrine','Hba1c','Fibrinogen','CRP', 'HDL')

# creating 1 df
ordinal_regression_df = inner_join(bp_df[,c(1,4)], subject_info_df[,4:8]) %>%
    # adding allostatic load data
    inner_join(mediator_scores_df) %>%
    # obtaining allostatic load biomarkers only
    #filter(Variable %in% allostatic_load_biomarkers) %>%
    pivot_wider(names_from = Variable, values_from = Mediator_Score)

# turning BP into a factor
ordinal_regression_df$BP_Classification = factor(ordinal_regression_df$BP_Classification, levels = c("N", "E", "1", '2'))

#reordering columns
ordinal_regression_df = ordinal_regression_df[,c(1,3:23,2)]
    

head(ordinal_regression_df)

Joining with `by = join_by(Subject_ID)`
Joining with `by = join_by(Subject_ID)`


Subject_ID,Group,Sex,Age,Race,Noradrenaline,Hba1c,Fibrinogen,CRP,IP10,⋯,IL13,IL1B,IL2,IL4,IL6,IL8,TNFa,HDL,Epinephrine,BP_Classification
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
CS_26,CS,F,34,B,0.00000000,0.1261076,0.35238170,0.03189947,0.07017161,⋯,0.3271404,0.00000000,2.351685e-07,0.25823172,0.18675526,0.001686759,0.14780720,0.01961938,0.09668427,1
CS_29,CS,F,36,B,0.09028609,0.3276962,0.18618128,0.04754286,0.03571227,⋯,0.7220417,0.01153174,1.419332e-08,0.53046857,0.43704343,0.355268670,0.08243793,0.29810436,0.48260060,N
CS_17,CS,M,29,W,0.32976117,0.2873836,0.18107212,0.01664812,0.12055087,⋯,0.3411684,0.00000000,6.761111e-08,0.49508420,0.03189408,0.147350214,0.26384236,0.52779975,0.72404411,N
NS_63,NS,M,43,B,0.25782803,0.4462655,0.11215630,0.22912585,0.42703285,⋯,0.6317240,0.00000000,1.132208e-06,0.60115086,1.00000000,0.119470619,0.42067290,0.13049306,0.67048546,E
CS_21,CS,F,31,B,0.27113435,0.7550424,0.16931582,0.12987698,0.35359516,⋯,0.2035670,0.00000000,9.701944e-07,0.07874693,0.30264289,0.418614119,0.26384236,0.06673031,0.31548045,N
CS_27,CS,F,35,W,0.23404945,0.3807725,0.03079665,0.02882542,0.02232779,⋯,0.4987997,0.00000000,7.491238e-08,0.35320369,0.14274251,0.033546943,0.38209701,0.14858426,0.00000000,1


In [11]:
# creating a vector of booleans that specifies whether or not the coefficient should be
# positive (TRUE) or negative (FALSE), only HDL will be negative
biomarker_coefficients = c(rep(TRUE, ncol(ordinal_regression_df) - 8), FALSE, TRUE)

estimateOrdWeights <- function(dataset, response, coefficients_magnitude){
    # ADD WORDS
    # ordinal regression or estimating ordinal weights
    
    ############# get required Regression Parameters
    
    #get number of response levels
    NoBins <- length(unique(dataset[[response]]))

    #get number of features or slieces
    noSlices <- ncol(dataset) - 2
    
    #run the regression
    feature_data <- as.matrix(dataset[,c(2:(ncol(dataset)-1))])
    #class(featuredata) <- "numeric"

    # ordinal regression
    # positiveID specifies whether or not each coefficient for each ind. variable should be constrained to be non-negative
    regression_model <- ordinalNet(feature_data, dataset[[response]], positiveID = coefficients_magnitude, 
                            #lambdaVals defines how quickly your coefficients drop to 0
                            # keeping it at 0 tries to minimize feature reduction
                            # maxiterout defines the number of iterations before it stops
                         lambdaVals = 0, maxiterOut = 100)
    testW <- regression_model$coefs[NoBins:(noSlices + NoBins - 1)]

    # rescaling the coefficients so that they sum to 1,  but double check!!!
    # add absolute value to make this work
    testW <- testW/sum(testW)

    # adding the variable names back in 
    test_weights_df = data.frame(Variable = colnames(feature_data), Coefficient = testW) %>%
        arrange(-Coefficient)
    
    return(test_weights_df)
}

# calling fn
estimateOrdWeights(ordinal_regression_df[,c(1,6:23)], 'BP_Classification', biomarker_coefficients)

Variable,Coefficient
<chr>,<dbl>
IL2,0.356268450
IL8,0.178089301
IL12p70,0.147449601
HDL,0.092405715
IL10,0.088587150
Epinephrine,0.055549307
IL13,0.040986936
Noradrenaline,0.032724287
IL4,0.007939252


Why wasn't HDL able to be constrained to negative?
What do the values mean? What considered high vs. low?
How does this first function relate to the second function below?

Run a linear regression to determine allostatic load score before looking at the chunk below. This will allow me to focus on the known samples and rank those using the coe